# cell 1 : import libaries

In [30]:
import pandas as pd
import numpy as np
import ast


In [31]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

merging Data set

In [24]:
movies = movies.merge(credits,on='title')

# Select only the columns we need
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Remove any rows with missing information
movies.dropna(inplace=True)


preprocessing

In [25]:
import ast # To safely evaluate string literals

# This function will help us extract the names
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

# Apply the function to the 'genres' and 'keywords' columns
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

# This function will get the top 3 actors from the 'cast'
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

movies['cast'] = movies['cast'].apply(convert3)

# This function will get only the director's name from the 'crew'
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['crew'] = movies['crew'].apply(fetch_director)

# Finally, let's split the 'overview' text into a list of words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

remove spaces 

In [26]:
# Remove spaces from all names to create unique entities
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# Combine all the processed lists into a single 'tags' column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

LAst step of data preprocessing , Final. dayaframe

In [27]:
# Create the final dataframe with just the necessary columns
new_df = movies[['movie_id', 'title', 'tags']].copy()

# Convert the 'tags' list into a single string
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

# Convert the entire 'tags' string to lowercase
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

The next step is the most important machine learning part. We need to convert the text in the tags column into numbers (vectors) so that the computer can understand and compare them. This is called Vectorization.

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the vectorizer
# max_features=5000 means we'll take the 5000 most common words
# stop_words='english' removes common English words like 'the', 'a', 'in'
cv = CountVectorizer(max_features=5000, stop_words='english')

# Transform the tags into vectors
vectors = cv.fit_transform(new_df['tags']).toarray()